# Reciprocal BLAST Hits (RBH)

**Author:** Paola Corbín Agustí, GitHub: paocorbin  
**Date:** December 15, 2023  


The `blast_rbh` function is adapted from the following source:

- Widdowquinn. "2018-03-06-ibioic/blob/master/02-sequence_databases/05-blast_for_rbh.ipynb." GitHub. Available at: [https://github.com/widdowquinn/2018-03-06-ibioic/blob/master/02-sequence_databases/05-blast_for_rbh.ipynb](https://github.com/widdowquinn/2018-03-06-ibioic/blob/master/02-sequence_databases/05-blast_for_rbh.ipynb).  

The original source notebook is part of the IBioIC Introduction to Bioinformatics Training Course Virtual Machine, which is available on Zenodo:

- Pritchard, L., Jones, S., & Cock, P. (2018). IBioIC Introduction to Bioinformatics Training Course Virtual Machine (March 2018). Zenodo. DOI: [10.5281/zenodo.1184095](https://doi.org/10.5281/zenodo.1184095)

This work is licensed under a Creative Commons Attribution 4.0 International License (CC-BY 4.0). Please note that this resource on Zenodo may provide access to virtual machine files rather than directly accessible code files.

In [1]:
#Import libraries
import hashlib
import os
import pandas as pd
from zeep import Client
# Import Biopython tools for running local BLASTX v2.9.0+
from Bio.Blast.Applications import NcbiblastpCommandline

In [2]:
#Working directory

path_enzymes = '../BRENDA/all/'
outdir_results = '../BRENDA/results/'

In [3]:
# Functions

def write_sequence(sequence, fgenoma):
    """
    Write a sequence entry to a fasta file
    Parameters:
    sequence (dict): Dictionary containing sequence information.
    fgenoma (file object): fasta file to write to.
    """
    sequence_header = '>' + sequence['firstAccessionCode'] + '|' + sequence['organism'].replace(' ', '_') + '|' + sequence['source'] + '\n'
    sequence_seq = sequence['sequence'] + '\n'
    fgenoma.write(sequence_header)
    fgenoma.write(sequence_seq)

    
def fasta_BRENDA_EC(ecnumber, path_out, reviewed=False):
    """
    Search sequences for a EC number in the BRENDA database 
    and save them to a fasta file.
    Parameters:
    ecnumber (str): The EC number to search for.
    path_out (str): Path to save the fasta file.
    reviewed (bool): If True, only include sequences reviewed in UniProt (Swiss-Prot).
    """
    # BRENDA API details
    wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
    user = "XXXXX@XXXXX" # Change XXXXX: password and user
    password = hashlib.sha256("XXXXXXXXXXXXXX".encode("utf-8")).hexdigest()
    client = Client(wsdl)
    
    # Set search parameters for BRENDA API
    ecnumber = "ecNumber*" + ecnumber
    parameters = (user, password, ecnumber, "sequence*", "noOfAminoAcids*", "firstAccessionCode*", "source*", "id*", "organism*")
    resultString = client.service.getSequence(*parameters)
    
    # Create fasta file
    fgenoma = open(path_out, 'w')
    for sequence in resultString:
        if reviewed==True: # Only include reviewed sequences
            if sequence['source'] == "Swiss-Prot":
                write_sequence(sequence, fgenoma)
        else:
            write_sequence(sequence, fgenoma)
    fgenoma.close()

    
def blast_rbh(outdir, s1, s2, EC):
    """
    Perform reciprocal BLAST hit (RBH) analysis between two sequence files.
    Parameters:
    outdir (str): Output directory for BLAST results.
    s1 (str): Path to the first sequence file (query).
    s2 (str): Path to the second sequence file (subject).
    EC (str): EC number being analyzed.

    This function is adapted from:
    Widdowquinn. "Teaching-IBioIC-Intro-to-Bioinformatics/02-sequence_databases/05-blast_for_rbh.ipynb." 
    GitHub, https://github.com/widdowquinn/Teaching-IBioIC-Intro-to-Bioinformatics/blob/master/02-sequence_databases/05-blast_for_rbh.ipynb.
    """
    # Create output directory
    os.makedirs(outdir, exist_ok=True)

    # Define output BLAST results filenames
    fwd_file = EC+'-fwd.tab'
    rev_file = EC+'-rev.tab'
    fwd_out = os.path.join(outdir, fwd_file)
    rev_out = os.path.join(outdir, rev_file)

    # Create BLAST command-lines for forward and reverse BLAST searches
    fwd_blastp = NcbiblastpCommandline(query=s1, subject=s2, out=fwd_out,
                               outfmt="6 qseqid sseqid pident qcovs qlen slen length bitscore evalue",
                               max_target_seqs=5, num_threads=8)
    rev_blastp = NcbiblastpCommandline(query=s2, subject=s1, out=rev_out,
                               outfmt="6 qseqid sseqid pident qcovs qlen slen length bitscore evalue",
                               max_target_seqs=5, num_threads=8)

    # Inspect command-lines
    print("FORWARD: %s" % fwd_blastp)
    print("REVERSE: %s" % rev_blastp)

    # Run BLAST searches
    # !! Uncomment to run local BLAST searches !!
    fwd_stdout, fwd_stderr = fwd_blastp()
    rev_stdout, rev_stderr = rev_blastp()

    # Load BLAST results into Pandas dataframes
    fwd_results = pd.read_csv(fwd_out, sep="\t", header=None)
    rev_results = pd.read_csv(rev_out, sep="\t", header=None)

    # Add headers to BLAST results dataframes
    headers = ["query", "subject", "identity", "coverage",
               "qlength", "slength", "alength",
               "bitscore", "E-value"]
    fwd_results.columns = headers
    rev_results.columns = headers

    # Calculate normalized bitscore in both dataframes 
    fwd_results['norm_bitscore'] = fwd_results.bitscore/fwd_results.qlength
    rev_results['norm_bitscore'] = rev_results.bitscore/rev_results.qlength

    # Create query and subject coverage columns
    fwd_results['qcov'] = fwd_results.alength/fwd_results.qlength
    rev_results['qcov'] = rev_results.alength/rev_results.qlength
    fwd_results['scov'] = fwd_results.alength/fwd_results.slength
    rev_results['scov'] = rev_results.alength/rev_results.slength

    # Clip maximum coverage values at 1.0
    fwd_results['qcov'] = fwd_results['qcov'].clip(upper=1)
    rev_results['qcov'] = rev_results['qcov'].clip(upper=1)
    fwd_results['scov'] = fwd_results['scov'].clip(upper=1)
    rev_results['scov'] = rev_results['scov'].clip(upper=1)

    # Merge forward and reverse results to find RBH
    rbbh = pd.merge(fwd_results, rev_results[['query', 'subject']],
                    left_on='subject', right_on='query',
                    how='outer')

    # Discard rows that are not RBH
    rbbh = rbbh.loc[rbbh.query_x == rbbh.subject_y]
    
    # Rename columns
    subject=s2.split('/')[-1].replace('.faa', '')
    name_cols = {'query_x': 'query_UniProt', 'subject_x': 'subject_' + subject}
    rbbh = rbbh.rename(columns=name_cols)
    
    # Add additional columns
    rbbh[['UniProt_ID', 'query_organism', 'query_source']] = rbbh['subject_y'].str.split('|', expand=True)
    rbbh['query_UniProt'] = rbbh['query_UniProt'].str.split('|').str[0]

    # Group duplicate RBH rows, taking the maximum value in each column
    rbbh = rbbh.groupby(['query_UniProt', 'subject_'+subject]).max()
    
    # Inspect the results
    display(rbbh.head(5))
    
    # Remove unnecessary columns
    cols_out = ['UniProt_ID', 'norm_bitscore', 'query_y', 'subject_y', 'qcov', 'scov']
    rbbh = rbbh.drop(cols_out, axis=1)

    # Inspect the results
    display(rbbh.head())

    # Filter results based on coverage, bitscore and E-value thresholds
    rbbh_filter = rbbh.loc[(rbbh['coverage']>=80) & (rbbh['bitscore']> 40) & (rbbh['E-value']>1*10^-6)]

    # Save results to excel files
    result_name = subject + '_' + EC
    filename = os.path.join(outdir, result_name)
    try:  
        rbbh_filter.to_excel(filename + '_filtered.xlsx')
    except: #if there is an empty rbbh_filter df
        open(filename + '_filtered.xlsx', 'w')
    rbbh.to_excel(filename + '.xlsx')

    
def ec_blast(path_enzymes, ecs, s2, outdir):
    """
    Perform BLAST searches for a list of EC numbers and save results.
    Parameters:
    path_enzymes (str): Path to the directory containing enzyme fasta files.
    ecs (list): List of EC numbers to search for.
    s2 (str): Path to the target sequence file.
    outdir (str): Output directory for the BLAST results.
    """
    for ec in ecs:
        ec = ec.replace('.', '_')
        s1 = path_enzymes + ec + '__BRENDA_sequences.fasta'

        if not os.path.isdir(outdir):
            command_line = 'mkdir ' + outdir
            os.system(command_line) 

        if s2 == '':
            open(outdir_results + ec + '_NOT_PRESENT.txt', 'a').close()
        else:
            if ec + '_filtered.xlsx' not in os.listdir(outdir):
                blast_rbh(outdir,s1,s2,ec)

## 1.1 Retrieve FASTA sequences from BRENDA 

### P3_M03 EC numbers to search (WLP)

In [10]:
ecs_P3_M03 = ['1.7.1.10', '6.3.4.3', '3.5.4.9',
      '1.5.1.5', '1.5.7.1', '1.5.1.54',
      '2.1.1.258', '1.2.7.4', '2.3.1.169']

# Retrieve all sequences in BRENDA for each EC number  
for ec in ecs_P3_M03:
    ecnumber = ec.replace('.', '_')
    path_out = path_enzymes + ecnumber + '__BRENDA_sequences.fasta'
    fasta_BRENDA_EC(ec, path_out, reviewed=False) 

### P4_M26 EC numbers to search (Methanogenesis)

In [11]:
ecs_P4_M26 = ['2.1.1.90', '2.1.1.246', '2.1.1.247',
      '2.1.1.248', '2.1.1.249', '2.1.1.250',
      '1.8.98.1', '2.8.4.1', '2.7.2.1',
      '2.3.1.8', '6.2.1.1', '2.3.1.169',
      '2.1.1.245', '1.2.7.4', '2.1.1.86'
      ]

# Retrieve all sequences in BRENDA for each EC number 
for ec in ecs_P4_M26:
    ecnumber = ec.replace('.', '_')
    path_out = path_enzymes + ecnumber + '__BRENDA_sequences.fasta'
    fasta_BRENDA_EC(ec, path_out, reviewed=False)

## 1.2 Reciprocal BLAST Hit

In [4]:
ecs_P3_M03 = ['1.7.1.10', '6.3.4.3', '3.5.4.9',
      '1.5.1.5', '1.5.1.54',
      '2.1.1.258', '1.2.7.4', '2.3.1.169']
# The enzyme '1.5.7.1' does not have sequences in BRENDA,
# so it is not included in ecs_P3_M03

# Perform reciprocal BLAST Hit against P3_M03
s2='../genomes/P3_M03.faa'
outdir=outdir_results+'P3_M03'
ec_blast(path_enzymes, ecs_P3_M03, s2, outdir)

FORWARD: blastp -out ../BRENDA/results/P3_M03/1_7_1_10-fwd.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../BRENDA/all/1_7_1_10__BRENDA_sequences.fasta -max_target_seqs 5 -num_threads 8 -subject ../genomes/P3_M03.faa
REVERSE: blastp -out ../BRENDA/results/P3_M03/1_7_1_10-rev.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../genomes/P3_M03.faa -max_target_seqs 5 -num_threads 8 -subject ../BRENDA/all/1_7_1_10__BRENDA_sequences.fasta


identity  coverage  qlength  slength  alength  \
query_UniProt subject_P3_M03                                                   
Q7MVY0        k141_193345_8      29.060      21.0    550.0    243.0    117.0   
              k141_346391_21     44.384      99.0    550.0    542.0    552.0   
              k141_387348_11     41.446      99.0    550.0    560.0    567.0   
              k141_525170_153    45.833       4.0    550.0    162.0     24.0   
              k141_525170_38     20.968      56.0    550.0    672.0    372.0   

                               bitscore        E-value  norm_bitscore  \
query_UniProt subject_P3_M03                                            
Q7MVY0        k141_193345_8        28.1   8.000000e-01       0.051091   
              k141_346391_21      451.0  3.120000e-155       0.820000   
              k141_387348_11      427.0  1.500000e-145       0.776364   
              k141_525170_153      26.2   3.100000e+00       0.047636   
              k141_525170_38       41.6   7.550000e-05       0.075636   

                                   qcov      scov          query_y  \
query_UniProt subject_P3_M03                                         
Q7MVY0        k141_193345_8    0.212727  0.481481    k141_193345_8   
              k141_346391_21   1.000000  1.000000   k141_346391_21   
              k141_387348_11   1.000000  1.000000   k141_387348_11   
              k141_525170_153  0.043636  0.148148  k141_525170_153   
              k141_525170_38   0.676364  0.553571   k141_525170_38   

                                                                       subject_y  \
query_UniProt subject_P3_M03                                                       
Q7MVY0        k141_193345_8    Q7MVY0|Porphyromonas_gingivalis_(strain_ATCC_B...   
              k141_346391_21   Q7MVY0|Porphyromonas_gingivalis_(strain_ATCC_B...   
              k141_387348_11   Q7MVY0|Porphyromonas_gingivalis_(strain_ATCC_B...   
              k141_525170_153  Q7MVY0|Porphyromonas_gingivalis_(strain_ATCC_B...   
              k141_525170_38   Q7MVY0|Porphyromonas_gingivalis_(strain_ATCC_B...   

                              UniProt_ID  \
query_UniProt subject_P3_M03               
Q7MVY0        k141_193345_8       Q7MVY0   
              k141_346391_21      Q7MVY0   
              k141_387348_11      Q7MVY0   
              k141_525170_153     Q7MVY0   
              k141_525170_38      Q7MVY0   

                                                                  query_organism  \
query_UniProt subject_P3_M03                                                       
Q7MVY0        k141_193345_8    Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   
              k141_346391_21   Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   
              k141_387348_11   Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   
              k141_525170_153  Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   
              k141_525170_38   Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   

                              query_source  
query_UniProt subject_P3_M03                
Q7MVY0        k141_193345_8     Swiss-Prot  
              k141_346391_21    Swiss-Prot  
              k141_387348_11    Swiss-Prot  
              k141_525170_153   Swiss-Prot  
              k141_525170_38    Swiss-Prot

identity  coverage  qlength  slength  alength  \
query_UniProt subject_P3_M03                                                   
Q7MVY0        k141_193345_8      29.060      21.0    550.0    243.0    117.0   
              k141_346391_21     44.384      99.0    550.0    542.0    552.0   
              k141_387348_11     41.446      99.0    550.0    560.0    567.0   
              k141_525170_153    45.833       4.0    550.0    162.0     24.0   
              k141_525170_38     20.968      56.0    550.0    672.0    372.0   

                               bitscore        E-value  \
query_UniProt subject_P3_M03                             
Q7MVY0        k141_193345_8        28.1   8.000000e-01   
              k141_346391_21      451.0  3.120000e-155   
              k141_387348_11      427.0  1.500000e-145   
              k141_525170_153      26.2   3.100000e+00   
              k141_525170_38       41.6   7.550000e-05   

                                                                  query_organism  \
query_UniProt subject_P3_M03                                                       
Q7MVY0        k141_193345_8    Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   
              k141_346391_21   Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   
              k141_387348_11   Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   
              k141_525170_153  Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   
              k141_525170_38   Porphyromonas_gingivalis_(strain_ATCC_BAA-308_...   

                              query_source  
query_UniProt subject_P3_M03                
Q7MVY0        k141_193345_8     Swiss-Prot  
              k141_346391_21    Swiss-Prot  
              k141_387348_11    Swiss-Prot  
              k141_525170_153   Swiss-Prot  
              k141_525170_38    Swiss-Prot

FORWARD: blastp -out ../BRENDA/results/P3_M03/6_3_4_3-fwd.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../BRENDA/all/6_3_4_3__BRENDA_sequences.fasta -max_target_seqs 5 -num_threads 8 -subject ../genomes/P3_M03.faa
REVERSE: blastp -out ../BRENDA/results/P3_M03/6_3_4_3-rev.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../genomes/P3_M03.faa -max_target_seqs 5 -num_threads 8 -subject ../BRENDA/all/6_3_4_3__BRENDA_sequences.fasta


,,identity,coverage,qlength,slength,alength,bitscore,E-value,norm_bitscore,qcov,scov,query_y,subject_y,UniProt_ID,query_organism,query_source
query_UniProt,subject_P3_M03,,,,,,,,,,,,,,,
A0A011QW40,k141_95372_6,27.000,16.0,556.0,348.0,100.0,30.0,0.280,0.053957,0.179856,0.287356,k141_95372_6,A0A011QW40|Alkalibacterium_sp._AK22|TrEMBL,A0A011QW40,Alkalibacterium_sp._AK22,TrEMBL
A0A016V8R2,k141_384909_31,28.571,15.0,589.0,432.0,91.0,29.3,0.530,0.049745,0.154499,0.210648,k141_384909_31,A0A016V8R2|Ancylostoma_ceylanicum|TrEMBL,A0A016V8R2,Ancylostoma_ceylanicum,TrEMBL
A0A016V9I2,k141_384909_31,28.571,14.0,643.0,432.0,91.0,28.9,0.680,0.044946,0.141524,0.210648,k141_384909_31,A0A016V9I2|Ancylostoma_ceylanicum|TrEMBL,A0A016V9I2,Ancylostoma_ceylanicum,TrEMBL
A0A017HA03,k141_145867_2,28.182,16.0,558.0,899.0,110.0,30.4,0.210,0.054480,0.197133,0.122358,k141_145867_2,A0A017HA03|Limimaricola_hongkongensis_DSM_1749...,A0A017HA03,Limimaricola_hongkongensis_DSM_17492,TrEMBL
A0A017STV9,k141_64912_35,36.538,9.0,563.0,508.0,52.0,32.7,0.043,0.058082,0.092362,0.102362,k141_64912_35,A0A017STV9|Chondromyces_apiculatus_DSM_436|TrEMBL,A0A017STV9,Chondromyces_apiculatus_DSM_436,TrEMBL


...

,,identity,coverage,qlength,slength,alength,bitscore,E-value,norm_bitscore,qcov,scov,query_y,subject_y,UniProt_ID,query_organism,query_source
query_UniProt,subject_P3_M03,,,,,,,,,,,,,,,
A0A011WTV8,k141_169253_3,45.833,3.0,707.0,343.0,24.0,29.3,0.530,0.041443,0.033946,0.069971,k141_169253_3,A0A011WTV8|Ruminococcus_albus_SY3|TrEMBL,A0A011WTV8,Ruminococcus_albus_SY3,TrEMBL
A0A031WG23,k141_375056_63,34.286,10.0,639.0,319.0,70.0,27.7,1.400,0.043349,0.109546,0.219436,k141_375056_63,A0A031WG23|Clostridioides_difficile|TrEMBL,A0A031WG23,Clostridioides_difficile,TrEMBL
A0A059VNZ5,k141_433621_13,25.301,13.0,636.0,410.0,83.0,29.3,0.580,0.046069,0.130503,0.202439,k141_433621_13,A0A059VNZ5|Citrobacter_amalonaticus_Y19|TrEMBL,A0A059VNZ5,Citrobacter_amalonaticus_Y19,TrEMBL
A0A060I744,k141_375056_68,28.302,33.0,159.0,421.0,53.0,27.3,0.350,0.171698,0.333333,0.125891,k141_375056_68,A0A060I744|Rhizobium_sp._IE4771|TrEMBL,A0A060I744,Rhizobium_sp._IE4771,TrEMBL
A0A062UXX5,k141_33655_14,37.333,11.0,663.0,226.0,75.0,32.3,0.041,0.048718,0.113122,0.331858,k141_33655_14,A0A062UXX5|Candidatus_Methanoperedens_nitrored...,A0A062UXX5,Candidatus_Methanoperedens_nitroreducens,TrEMBL


,,identity,coverage,qlength,slength,alength,bitscore,E-value,query_organism,query_source
query_UniProt,subject_P3_M03,,,,,,,,,
A0A011WTV8,k141_169253_3,45.833,3.0,707.0,343.0,24.0,29.3,0.530,Ruminococcus_albus_SY3,TrEMBL
A0A031WG23,k141_375056_63,34.286,10.0,639.0,319.0,70.0,27.7,1.400,Clostridioides_difficile,TrEMBL
A0A059VNZ5,k141_433621_13,25.301,13.0,636.0,410.0,83.0,29.3,0.580,Citrobacter_amalonaticus_Y19,TrEMBL
A0A060I744,k141_375056_68,28.302,33.0,159.0,421.0,53.0,27.3,0.350,Rhizobium_sp._IE4771,TrEMBL
A0A062UXX5,k141_33655_14,37.333,11.0,663.0,226.0,75.0,32.3,0.041,Candidatus_Methanoperedens_nitroreducens,TrEMBL


FORWARD: blastp -out ../BRENDA/results/P3_M03/2_3_1_169-fwd.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../BRENDA/all/2_3_1_169__BRENDA_sequences.fasta -max_target_seqs 5 -num_threads 8 -subject ../genomes/P3_M03.faa
REVERSE: blastp -out ../BRENDA/results/P3_M03/2_3_1_169-rev.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../genomes/P3_M03.faa -max_target_seqs 5 -num_threads 8 -subject ../BRENDA/all/2_3_1_169__BRENDA_sequences.fasta


identity  coverage  qlength  slength  alength  \
query_UniProt subject_P3_M03                                                  
A0A031WFW6    k141_3496_10      21.384      21.0    708.0    233.0    159.0   
A0A062VBT1    k141_202602_2     32.787      25.0    471.0   1000.0     84.0   
              k141_346391_22    42.857       9.0    471.0    364.0     49.0   
A0A075WVS6    k141_196736_4     35.897       5.0    738.0    296.0     39.0   
              k141_346391_27    30.986       9.0    738.0    251.0     71.0   

                              bitscore  E-value  norm_bitscore      qcov  \
query_UniProt subject_P3_M03                                               
A0A031WFW6    k141_3496_10        28.5    0.910       0.040254  0.224576   
A0A062VBT1    k141_202602_2       30.4    2.500       0.064544  0.178344   
              k141_346391_22      26.6    2.900       0.056476  0.104034   
A0A075WVS6    k141_196736_4       28.5    0.970       0.038618  0.052846   
              k141_346391_27      32.0    0.073       0.043360  0.096206   

                                  scov         query_y  \
query_UniProt subject_P3_M03                             
A0A031WFW6    k141_3496_10    0.682403    k141_3496_10   
A0A062VBT1    k141_202602_2   0.084000   k141_202602_2   
              k141_346391_22  0.134615  k141_346391_22   
A0A075WVS6    k141_196736_4   0.131757   k141_196736_4   
              k141_346391_27  0.282869  k141_346391_27   

                                                                      subject_y  \
query_UniProt subject_P3_M03                                                      
A0A031WFW6    k141_3496_10           A0A031WFW6|Clostridioides_difficile|TrEMBL   
A0A062VBT1    k141_202602_2   A0A062VBT1|Candidatus_Methanoperedens_nitrored...   
              k141_346391_22  A0A062VBT1|Candidatus_Methanoperedens_nitrored...   
A0A075WVS6    k141_196736_4   A0A075WVS6|Thermodesulfobacterium_commune_DSM_...   
              k141_346391_27  A0A075WVS6|Thermodesulfobacterium_commune_DSM_...   

                              UniProt_ID  \
query_UniProt subject_P3_M03               
A0A031WFW6    k141_3496_10    A0A031WFW6   
A0A062VBT1    k141_202602_2   A0A062VBT1   
              k141_346391_22  A0A062VBT1   
A0A075WVS6    k141_196736_4   A0A075WVS6   
              k141_346391_27  A0A075WVS6   

                                                        query_organism  \
query_UniProt subject_P3_M03                                             
A0A031WFW6    k141_3496_10                    Clostridioides_difficile   
A0A062VBT1    k141_202602_2   Candidatus_Methanoperedens_nitroreducens   
              k141_346391_22  Candidatus_Methanoperedens_nitroreducens   
A0A075WVS6    k141_196736_4    Thermodesulfobacterium_commune_DSM_2178   
              k141_346391_27   Thermodesulfobacterium_commune_DSM_2178   

                             query_source  
query_UniProt subject_P3_M03               
A0A031WFW6    k141_3496_10         TrEMBL  
A0A062VBT1    k141_202602_2        TrEMBL  
              k141_346391_22       TrEMBL  
A0A075WVS6    k141_196736_4        TrEMBL  
              k141_346391_27       TrEMBL

identity  coverage  qlength  slength  alength  \
query_UniProt subject_P3_M03                                                  
A0A031WFW6    k141_3496_10      21.384      21.0    708.0    233.0    159.0   
A0A062VBT1    k141_202602_2     32.787      25.0    471.0   1000.0     84.0   
              k141_346391_22    42.857       9.0    471.0    364.0     49.0   
A0A075WVS6    k141_196736_4     35.897       5.0    738.0    296.0     39.0   
              k141_346391_27    30.986       9.0    738.0    251.0     71.0   

                              bitscore  E-value  \
query_UniProt subject_P3_M03                      
A0A031WFW6    k141_3496_10        28.5    0.910   
A0A062VBT1    k141_202602_2       30.4    2.500   
              k141_346391_22      26.6    2.900   
A0A075WVS6    k141_196736_4       28.5    0.970   
              k141_346391_27      32.0    0.073   

                                                        query_organism  \
query_UniProt subject_P3_M03                                             
A0A031WFW6    k141_3496_10                    Clostridioides_difficile   
A0A062VBT1    k141_202602_2   Candidatus_Methanoperedens_nitroreducens   
              k141_346391_22  Candidatus_Methanoperedens_nitroreducens   
A0A075WVS6    k141_196736_4    Thermodesulfobacterium_commune_DSM_2178   
              k141_346391_27   Thermodesulfobacterium_commune_DSM_2178   

                             query_source  
query_UniProt subject_P3_M03               
A0A031WFW6    k141_3496_10         TrEMBL  
A0A062VBT1    k141_202602_2        TrEMBL  
              k141_346391_22       TrEMBL  
A0A075WVS6    k141_196736_4        TrEMBL  
              k141_346391_27       TrEMBL

In [6]:
ecs_P4_M26 = ['2.1.1.90', '2.1.1.246', '2.1.1.247',
      '2.1.1.248', '2.1.1.249', '2.1.1.250',
      '1.8.98.1', '2.8.4.1', '2.7.2.1',
      '2.3.1.8', '6.2.1.1', '2.3.1.169',
      '2.1.1.245', '1.2.7.4', '2.1.1.86'
      ]

# Perform reciprocal BLAST hit against P3_M03
s2='../genomes/P4_M26.faa'
outdir=outdir_results+'P4_M26'
ec_blast(path_enzymes, ecs_P4_M26, s2, outdir)

FORWARD: blastp -out ../BRENDA/results/P4_M26/2_1_1_90-fwd.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../BRENDA/all/2_1_1_90__BRENDA_sequences.fasta -max_target_seqs 5 -num_threads 8 -subject ../genomes/P4_M26.faa
REVERSE: blastp -out ../BRENDA/results/P4_M26/2_1_1_90-rev.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../genomes/P4_M26.faa -max_target_seqs 5 -num_threads 8 -subject ../BRENDA/all/2_1_1_90__BRENDA_sequences.fasta


identity  coverage  qlength  slength  alength  \
query_UniProt subject_P4_M26                                                   
A0A0A7LB62    k141_228794_21     44.444       6.0    471.0    135.0     27.0   
              k141_228794_96     42.500       8.0    471.0    316.0     40.0   
              k141_301924_148    25.714      14.0    471.0    588.0     70.0   
              k141_733366_186    73.517     100.0    471.0    467.0    472.0   
              k141_733366_283    28.125       7.0    471.0    869.0     32.0   

                               bitscore  E-value  norm_bitscore      qcov  \
query_UniProt subject_P4_M26                                                
A0A0A7LB62    k141_228794_21       26.2      1.2       0.055626  0.057325   
              k141_228794_96       26.6      1.5       0.056476  0.084926   
              k141_301924_148      26.2      2.0       0.055626  0.148620   
              k141_733366_186     730.0      0.0       1.549894  1.000000   
              k141_733366_283      25.4      4.1       0.053928  0.067941   

                                   scov          query_y  \
query_UniProt subject_P4_M26                               
A0A0A7LB62    k141_228794_21   0.200000   k141_228794_21   
              k141_228794_96   0.126582   k141_228794_96   
              k141_301924_148  0.119048  k141_301924_148   
              k141_733366_186  1.000000  k141_733366_186   
              k141_733366_283  0.036824  k141_733366_283   

                                                                       subject_y  \
query_UniProt subject_P4_M26                                                       
A0A0A7LB62    k141_228794_21   A0A0A7LB62|Candidatus_Methanoplasma_termitum|T...   
              k141_228794_96   A0A0A7LB62|Candidatus_Methanoplasma_termitum|T...   
              k141_301924_148  A0A0A7LB62|Candidatus_Methanoplasma_termitum|T...   
              k141_733366_186  A0A0A7LB62|Candidatus_Methanoplasma_termitum|T...   
              k141_733366_283  A0A0A7LB62|Candidatus_Methanoplasma_termitum|T...   

                               UniProt_ID                     query_organism  \
query_UniProt subject_P4_M26                                                   
A0A0A7LB62    k141_228794_21   A0A0A7LB62  Candidatus_Methanoplasma_termitum   
              k141_228794_96   A0A0A7LB62  Candidatus_Methanoplasma_termitum   
              k141_301924_148  A0A0A7LB62  Candidatus_Methanoplasma_termitum   
              k141_733366_186  A0A0A7LB62  Candidatus_Methanoplasma_termitum   
              k141_733366_283  A0A0A7LB62  Candidatus_Methanoplasma_termitum   

                              query_source  
query_UniProt subject_P4_M26                
A0A0A7LB62    k141_228794_21        TrEMBL  
              k141_228794_96        TrEMBL  
              k141_301924_148       TrEMBL  
              k141_733366_186       TrEMBL  
              k141_733366_283       TrEMBL

identity  coverage  qlength  slength  alength  \
query_UniProt subject_P4_M26                                                   
A0A0A7LB62    k141_228794_21     44.444       6.0    471.0    135.0     27.0   
              k141_228794_96     42.500       8.0    471.0    316.0     40.0   
              k141_301924_148    25.714      14.0    471.0    588.0     70.0   
              k141_733366_186    73.517     100.0    471.0    467.0    472.0   
              k141_733366_283    28.125       7.0    471.0    869.0     32.0   

                               bitscore  E-value  \
query_UniProt subject_P4_M26                       
A0A0A7LB62    k141_228794_21       26.2      1.2   
              k141_228794_96       26.6      1.5   
              k141_301924_148      26.2      2.0   
              k141_733366_186     730.0      0.0   
              k141_733366_283      25.4      4.1   

                                                  query_organism query_source  
query_UniProt subject_P4_M26                                                   
A0A0A7LB62    k141_228794_21   Candidatus_Methanoplasma_termitum       TrEMBL  
              k141_228794_96   Candidatus_Methanoplasma_termitum       TrEMBL  
              k141_301924_148  Candidatus_Methanoplasma_termitum       TrEMBL  
              k141_733366_186  Candidatus_Methanoplasma_termitum       TrEMBL  
              k141_733366_283  Candidatus_Methanoplasma_termitum       TrEMBL

FORWARD: blastp -out ../BRENDA/results/P4_M26/2_1_1_246-fwd.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../BRENDA/all/2_1_1_246__BRENDA_sequences.fasta -max_target_seqs 5 -num_threads 8 -subject ../genomes/P4_M26.faa
REVERSE: blastp -out ../BRENDA/results/P4_M26/2_1_1_246-rev.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../genomes/P4_M26.faa -max_target_seqs 5 -num_threads 8 -subject ../BRENDA/all/2_1_1_246__BRENDA_sequences.fasta


identity  coverage  qlength  slength  alength  \
query_UniProt subject_P4_M26                                                   
A0A069RG61    k141_301924_145    26.761      98.0    349.0    349.0    355.0   
              k141_301924_146    25.496      98.0    349.0    349.0    353.0   
              k141_527625_388    27.405      97.0    349.0    340.0    343.0   
A0A069RGP9    k141_301924_145    26.286      95.0    357.0    349.0    350.0   
              k141_301924_146    25.852      95.0    357.0    349.0    352.0   

                               bitscore       E-value  norm_bitscore  \
query_UniProt subject_P4_M26                                           
A0A069RG61    k141_301924_145     125.0  2.350000e-34       0.358166   
              k141_301924_146     112.0  1.260000e-29       0.320917   
              k141_527625_388     114.0  1.540000e-30       0.326648   
A0A069RGP9    k141_301924_145     105.0  4.530000e-27       0.294118   
              k141_301924_146     111.0  3.400000e-29       0.310924   

                                   qcov  scov          query_y  \
query_UniProt subject_P4_M26                                     
A0A069RG61    k141_301924_145  1.000000   1.0  k141_301924_145   
              k141_301924_146  1.000000   1.0  k141_301924_146   
              k141_527625_388  0.982808   1.0  k141_527625_388   
A0A069RGP9    k141_301924_145  0.980392   1.0  k141_301924_145   
              k141_301924_146  0.985994   1.0  k141_301924_146   

                                                                       subject_y  \
query_UniProt subject_P4_M26                                                       
A0A069RG61    k141_301924_145  A0A069RG61|Peptoclostridium_litorale_DSM_5388|...   
              k141_301924_146  A0A069RG61|Peptoclostridium_litorale_DSM_5388|...   
              k141_527625_388  A0A069RG61|Peptoclostridium_litorale_DSM_5388|...   
A0A069RGP9    k141_301924_145  A0A069RGP9|Peptoclostridium_litorale_DSM_5388|...   
              k141_301924_146  A0A069RGP9|Peptoclostridium_litorale_DSM_5388|...   

                               UniProt_ID                      query_organism  \
query_UniProt subject_P4_M26                                                    
A0A069RG61    k141_301924_145  A0A069RG61  Peptoclostridium_litorale_DSM_5388   
              k141_301924_146  A0A069RG61  Peptoclostridium_litorale_DSM_5388   
              k141_527625_388  A0A069RG61  Peptoclostridium_litorale_DSM_5388   
A0A069RGP9    k141_301924_145  A0A069RGP9  Peptoclostridium_litorale_DSM_5388   
              k141_301924_146  A0A069RGP9  Peptoclostridium_litorale_DSM_5388   

                              query_source  
query_UniProt subject_P4_M26                
A0A069RG61    k141_301924_145       TrEMBL  
              k141_301924_146       TrEMBL  
              k141_527625_388       TrEMBL  
A0A069RGP9    k141_301924_145       TrEMBL  
              k141_301924_146       TrEMBL

...

identity  coverage  qlength  slength  alength  \
query_UniProt subject_P4_M26                                                   
A0A021X303    k141_28886_4       27.536       7.0   1040.0   1886.0     69.0   
              k141_733366_32     31.250       8.0   1040.0   2457.0     96.0   
A0A031WG23    k141_301924_75     26.168      17.0    639.0    509.0    107.0   
A0A059MR94    k141_228794_107    34.375      11.0    293.0    167.0     32.0   
A0A062UXX5    k141_527625_300    31.373       8.0    663.0    228.0     51.0   

                               bitscore  E-value  norm_bitscore      qcov  \
query_UniProt subject_P4_M26                                                
A0A021X303    k141_28886_4         32.3    0.078       0.031058  0.066346   
              k141_733366_32       33.5    0.035       0.032212  0.092308   
A0A031WG23    k141_301924_75       27.7    1.000       0.043349  0.167449   
A0A059MR94    k141_228794_107      25.0    2.100       0.085324  0.109215   
A0A062UXX5    k141_527625_300      25.8    2.900       0.038914  0.076923   

                                   scov          query_y  \
query_UniProt subject_P4_M26                               
A0A021X303    k141_28886_4     0.036585     k141_28886_4   
              k141_733366_32   0.039072   k141_733366_32   
A0A031WG23    k141_301924_75   0.210216   k141_301924_75   
A0A059MR94    k141_228794_107  0.191617  k141_228794_107   
A0A062UXX5    k141_527625_300  0.223684  k141_527625_300   

                                                                       subject_y  \
query_UniProt subject_P4_M26                                                       
A0A021X303    k141_28886_4                   A0A021X303|Shinella_sp._DD12|TrEMBL   
              k141_733366_32                 A0A021X303|Shinella_sp._DD12|TrEMBL   
A0A031WG23    k141_301924_75          A0A031WG23|Clostridioides_difficile|TrEMBL   
A0A059MR94    k141_228794_107        A0A059MR94|Rhodococcus_aetherivorans|TrEMBL   
A0A062UXX5    k141_527625_300  A0A062UXX5|Candidatus_Methanoperedens_nitrored...   

                               UniProt_ID  \
query_UniProt subject_P4_M26                
A0A021X303    k141_28886_4     A0A021X303   
              k141_733366_32   A0A021X303   
A0A031WG23    k141_301924_75   A0A031WG23   
A0A059MR94    k141_228794_107  A0A059MR94   
A0A062UXX5    k141_527625_300  A0A062UXX5   

                                                         query_organism  \
query_UniProt subject_P4_M26                                              
A0A021X303    k141_28886_4                            Shinella_sp._DD12   
              k141_733366_32                          Shinella_sp._DD12   
A0A031WG23    k141_301924_75                   Clostridioides_difficile   
A0A059MR94    k141_228794_107                 Rhodococcus_aetherivorans   
A0A062UXX5    k141_527625_300  Candidatus_Methanoperedens_nitroreducens   

                              query_source  
query_UniProt subject_P4_M26                
A0A021X303    k141_28886_4          TrEMBL  
              k141_733366_32        TrEMBL  
A0A031WG23    k141_301924_75        TrEMBL  
A0A059MR94    k141_228794_107       TrEMBL  
A0A062UXX5    k141_527625_300       TrEMBL

identity  coverage  qlength  slength  alength  \
query_UniProt subject_P4_M26                                                   
A0A021X303    k141_28886_4       27.536       7.0   1040.0   1886.0     69.0   
              k141_733366_32     31.250       8.0   1040.0   2457.0     96.0   
A0A031WG23    k141_301924_75     26.168      17.0    639.0    509.0    107.0   
A0A059MR94    k141_228794_107    34.375      11.0    293.0    167.0     32.0   
A0A062UXX5    k141_527625_300    31.373       8.0    663.0    228.0     51.0   

                               bitscore  E-value  \
query_UniProt subject_P4_M26                       
A0A021X303    k141_28886_4         32.3    0.078   
              k141_733366_32       33.5    0.035   
A0A031WG23    k141_301924_75       27.7    1.000   
A0A059MR94    k141_228794_107      25.0    2.100   
A0A062UXX5    k141_527625_300      25.8    2.900   

                                                         query_organism  \
query_UniProt subject_P4_M26                                              
A0A021X303    k141_28886_4                            Shinella_sp._DD12   
              k141_733366_32                          Shinella_sp._DD12   
A0A031WG23    k141_301924_75                   Clostridioides_difficile   
A0A059MR94    k141_228794_107                 Rhodococcus_aetherivorans   
A0A062UXX5    k141_527625_300  Candidatus_Methanoperedens_nitroreducens   

                              query_source  
query_UniProt subject_P4_M26                
A0A021X303    k141_28886_4          TrEMBL  
              k141_733366_32        TrEMBL  
A0A031WG23    k141_301924_75        TrEMBL  
A0A059MR94    k141_228794_107       TrEMBL  
A0A062UXX5    k141_527625_300       TrEMBL

FORWARD: blastp -out ../BRENDA/results/P4_M26/2_1_1_86-fwd.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../BRENDA/all/2_1_1_86__BRENDA_sequences.fasta -max_target_seqs 5 -num_threads 8 -subject ../genomes/P4_M26.faa
REVERSE: blastp -out ../BRENDA/results/P4_M26/2_1_1_86-rev.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query ../genomes/P4_M26.faa -max_target_seqs 5 -num_threads 8 -subject ../BRENDA/all/2_1_1_86__BRENDA_sequences.fasta


identity  coverage  qlength  slength  alength  \
query_UniProt subject_P4_M26                                                   
A0A062UZX9    k141_733366_235    31.081      25.0    293.0    305.0     74.0   
A0A062UZY6    k141_301924_155    36.364      33.0     67.0    211.0     22.0   
              k141_527625_358    29.167      64.0     67.0    212.0     48.0   
A0A062V557    k141_527625_63     31.667      23.0    240.0    986.0     60.0   
A0A062V7D1    k141_527625_342    35.849      20.0    260.0    437.0     53.0   

                               bitscore  E-value  norm_bitscore      qcov  \
query_UniProt subject_P4_M26                                                
A0A062UZX9    k141_733366_235      25.8     1.40       0.088055  0.252560   
A0A062UZY6    k141_301924_155      22.3     1.30       0.332836  0.328358   
              k141_527625_358      23.1     0.61       0.344776  0.716418   
A0A062V557    k141_527625_63       25.8     1.30       0.107500  0.250000   
A0A062V7D1    k141_527625_342      25.4     1.80       0.097692  0.203846   

                                   scov          query_y  \
query_UniProt subject_P4_M26                               
A0A062UZX9    k141_733366_235  0.242623  k141_733366_235   
A0A062UZY6    k141_301924_155  0.104265  k141_301924_155   
              k141_527625_358  0.226415  k141_527625_358   
A0A062V557    k141_527625_63   0.060852   k141_527625_63   
A0A062V7D1    k141_527625_342  0.121281  k141_527625_342   

                                                                       subject_y  \
query_UniProt subject_P4_M26                                                       
A0A062UZX9    k141_733366_235  A0A062UZX9|Candidatus_Methanoperedens_nitrored...   
A0A062UZY6    k141_301924_155  A0A062UZY6|Candidatus_Methanoperedens_nitrored...   
              k141_527625_358  A0A062UZY6|Candidatus_Methanoperedens_nitrored...   
A0A062V557    k141_527625_63   A0A062V557|Candidatus_Methanoperedens_nitrored...   
A0A062V7D1    k141_527625_342  A0A062V7D1|Candidatus_Methanoperedens_nitrored...   

                               UniProt_ID  \
query_UniProt subject_P4_M26                
A0A062UZX9    k141_733366_235  A0A062UZX9   
A0A062UZY6    k141_301924_155  A0A062UZY6   
              k141_527625_358  A0A062UZY6   
A0A062V557    k141_527625_63   A0A062V557   
A0A062V7D1    k141_527625_342  A0A062V7D1   

                                                         query_organism  \
query_UniProt subject_P4_M26                                              
A0A062UZX9    k141_733366_235  Candidatus_Methanoperedens_nitroreducens   
A0A062UZY6    k141_301924_155  Candidatus_Methanoperedens_nitroreducens   
              k141_527625_358  Candidatus_Methanoperedens_nitroreducens   
A0A062V557    k141_527625_63   Candidatus_Methanoperedens_nitroreducens   
A0A062V7D1    k141_527625_342  Candidatus_Methanoperedens_nitroreducens   

                              query_source  
query_UniProt subject_P4_M26                
A0A062UZX9    k141_733366_235       TrEMBL  
A0A062UZY6    k141_301924_155       TrEMBL  
              k141_527625_358       TrEMBL  
A0A062V557    k141_527625_63        TrEMBL  
A0A062V7D1    k141_527625_342       TrEMBL

identity  coverage  qlength  slength  alength  \
query_UniProt subject_P4_M26                                                   
A0A062UZX9    k141_733366_235    31.081      25.0    293.0    305.0     74.0   
A0A062UZY6    k141_301924_155    36.364      33.0     67.0    211.0     22.0   
              k141_527625_358    29.167      64.0     67.0    212.0     48.0   
A0A062V557    k141_527625_63     31.667      23.0    240.0    986.0     60.0   
A0A062V7D1    k141_527625_342    35.849      20.0    260.0    437.0     53.0   

                               bitscore  E-value  \
query_UniProt subject_P4_M26                       
A0A062UZX9    k141_733366_235      25.8     1.40   
A0A062UZY6    k141_301924_155      22.3     1.30   
              k141_527625_358      23.1     0.61   
A0A062V557    k141_527625_63       25.8     1.30   
A0A062V7D1    k141_527625_342      25.4     1.80   

                                                         query_organism  \
query_UniProt subject_P4_M26                                              
A0A062UZX9    k141_733366_235  Candidatus_Methanoperedens_nitroreducens   
A0A062UZY6    k141_301924_155  Candidatus_Methanoperedens_nitroreducens   
              k141_527625_358  Candidatus_Methanoperedens_nitroreducens   
A0A062V557    k141_527625_63   Candidatus_Methanoperedens_nitroreducens   
A0A062V7D1    k141_527625_342  Candidatus_Methanoperedens_nitroreducens   

                              query_source  
query_UniProt subject_P4_M26                
A0A062UZX9    k141_733366_235       TrEMBL  
A0A062UZY6    k141_301924_155       TrEMBL  
              k141_527625_358       TrEMBL  
A0A062V557    k141_527625_63        TrEMBL  
A0A062V7D1    k141_527625_342       TrEMBL